## Loading the libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

Using TensorFlow backend.


## Exploring the dataset

In [2]:
data = [] 
labels = [] 
classes = 43
cur_path = os.getcwd()

for i in range(classes):
    path = os.path.join(cur_path,'Train',str(i))
    images = os.listdir(path)
    
    for a in images:
        try:
            image = Image.open(path + '\\'+ a)
            image = image.resize((30,30))
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error:Image Not Loaded")

data = np.array(data)
labels = np.array(labels)

## Splitting the dataset into training and testing

In [3]:
print(data.shape,labels.shape)
X_train, X_test, y_train,y_test = train_test_split(data,labels,test_size=0.2,random_state=42)

print(X_train.shape, X_test.shape, y_train.shape,y_test.shape)

y_train = to_categorical(y_train,43)
y_test = to_categorical(y_test,43)

(39209, 30, 30, 3) (39209,)
(31367, 30, 30, 3) (7842, 30, 30, 3) (31367,) (7842,)


## Building the CNN Model

In [8]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3,3),activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

#Compiling the model with adam optimizer
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

W1210 13:04:15.195527  7860 deprecation_wrapper.py:119] From H:\Anaconda\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1210 13:04:15.242399  7860 deprecation_wrapper.py:119] From H:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



## Training and Validating the model

In [9]:
epochs = 15

history = model.fit(X_train, y_train, batch_size=64, epochs=epochs, validation_data=(X_test,y_test))

W1210 13:05:47.243101  7860 deprecation.py:323] From H:\Anaconda\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 31367 samples, validate on 7842 samples
Epoch 1/15
31367/31367 [==============================] - 130s 4ms/step - loss: 2.1595 - acc: 0.4681 - val_loss: 0.6048 - val_acc: 0.8592
Epoch 2/15
31367/31367 [==============================] - 125s 4ms/step - loss: 0.6814 - acc: 0.7956 - val_loss: 0.2521 - val_acc: 0.9361
Epoch 3/15
31367/31367 [==============================] - 125s 4ms/step - loss: 0.4319 - acc: 0.8689 - val_loss: 0.1342 - val_acc: 0.9661
Epoch 4/15
31367/31367 [==============================] - 126s 4ms/step - loss: 0.3239 - acc: 0.9016 - val_loss: 0.1689 - val_acc: 0.9565
Epoch 5/15
31367/31367 [==============================] - 125s 4ms/step - loss: 0.2649 - acc: 0.9218 - val_loss: 0.0862 - val_acc: 0.9759
Epoch 6/15
31367/31367 [==============================] - 126s 4ms/step - loss: 0.2353 - acc: 0.9305 - val_loss: 0.0769 - val_acc: 0.9786
Epoch 7/15
31367/31367 [==============================] - 126s 4ms/step - loss: 0.2099 - acc: 0.9400 - val_loss: 0.0839 - v

**The model has resulted in an accuracy of 98% on the training dataset**

### Testing the model with test dataset

In [13]:
from sklearn.metrics import accuracy_score
import pandas as pd
y_test = pd.read_csv('Test.csv')

labels = y_test["ClassId"].values
imgs = y_test["Path"].values

data=[]

for img in imgs:
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))
    
X_test = np.array(data)

pred = model.predict_classes(X_test)

#Accuracy with test data
from sklearn.metrics import accuracy_score
accuracy_score(labels,pred)

0.9535233570863024

## Saving the model

In [14]:
model.save('traffic_classifier.h5')